## Example of Batch Processing Using Folder Name with Variable Tags

This example demonstrates how to batch process images using folder name as multiple variable tags. Each folder represents a different set of experimental conditions, and these conditions are extracted from the folder name as variables.Each variable intended to be represented a value in the specified column in the resulting output table. This variables in the name of the folder are delineated by underscores (`_`). The script then used the function to processes paired images within the folder and aggregates the results into master DataFrames, which are then saved as CSV files.

## Explanation

1. **Setup Output Directories**:
    - The script first sets up directories for storing the output data. It creates an `Outputs` folder and a subfolder `Output_images` to store the results and images respectively.

2. **Initialize Data Structures**:
    - Master DataFrames (`master_results` and `master_reports`) and other necessary data structures are initialized to collect the results from each processed folder.

3. **Iterate Over Folders**:
    - The script iterates over each subfolder in the `global_path` directory. Each subfolder represents a different date of experiments.

4. **Process Each Subfolder**:
    - For each subfolder (representing a specific experiment), it extracts variables (such as cells, treatment, concentration) from the folder name. These variables are expected to be separated by underscores (`_`).

5. **Process Paired Images**:
    - The function `in_folder_loop` is called to process the images within each subfolder. It uses the selected analysis function to generate results and reports.

6. **Assign Identifiers**:
    - The extracted variables (cells, treatment, concentration, date) and the name of the selected analysis function are added as columns to the results and reports DataFrames.




In [1]:
import pandas as pd
from pathlib import Path
import natsort
from FLIM_functions import analyze_single_fitted_channel,generate_small_report_fig,analyze_single_fastflim_channel
from helper_functions import in_folder_loop

In [9]:
global_path=Path(r"G:\SP8_FLIM\Treatments\Aldometanib")

In [10]:
selected_analysis_function = analyze_single_fitted_channel

In [20]:
import pandas as pd
from pathlib import Path
import natsort


# Create the folder where all the outputs will be collected 
output_folder_path = global_path / 'Outputs'
output_folder_path.mkdir(exist_ok=True)

outputimage_folder_path = output_folder_path / 'Output_images'
outputimage_folder_path.mkdir(exist_ok=True)

# Initialize master DataFrames and other data structures outside the loop
master_results = pd.DataFrame([])
master_reports = pd.DataFrame([])
report_images = []


# Iterate over all folders in the global_path directory
for day_path in global_path.iterdir():
    if day_path.is_dir() and day_path.stem != 'Outputs':
        
        date = day_path.stem
        print(f"Processing date folder: {date}")
        
        result_per_date = pd.DataFrame()
        
        # level of the folder with analisys, each variable is a 'tag' between _ 
        #For different experiment the columns may be modified depending on the design 
        
        for cell_treatment_path in day_path.iterdir():
            if cell_treatment_path.is_dir(): 
                
                cell_treatment = cell_treatment_path.stem
                
                # Split the folder name to extract cells, age, treatment, and concentration
                try:
                    cells, treatment, concentration = cell_treatment.split('_')
                
                except ValueError:
                    print(f"Skipping folder with unexpected name format: {cell_treatment}")
                    continue
                
                print(f"Cells: {cells}")
                print(f"Treatment: {treatment}")
                print(f"Concentration: {concentration}")
                print(f"  Processing cell_treatment folder: {cell_treatment}")

                result_per_folder,report_per_folder=in_folder_loop(cell_treatment_path,selected_analysis_function)
                
                # identifiction 
                result_per_folder['cells'] = cells
                result_per_folder['treatment'] = treatment
                result_per_folder['concentration'] = concentration

                result_per_folder['Date'] = date
                result_per_folder['selected_analysis'] = selected_analysis_function.__name__

                report_per_folder['cells'] = cells
                report_per_folder['treatment'] = treatment
                report_per_folder['concentration'] = concentration

                report_per_folder['Date'] = date
                report_per_folder['selected_analysis'] =selected_analysis_function.__name__

                # Concatenate the local table to the master DataFrame
                master_results = pd.concat([master_results, result_per_folder])
                master_reports = pd.concat([master_reports, report_per_folder])



# Save the master DataFrame as a CSV file
master_results.to_csv( "master_table.csv")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv("master_reports.csv")


Processing date folder: 2024-02-07
Cells: wt
Treatment: Aldo
Concentration: 0
  Processing cell_treatment folder: wt_Aldo_0
Cells: wt
Treatment: Aldo
Concentration: 1000
  Processing cell_treatment folder: wt_Aldo_1000
Processing date folder: 2024-02-02
Cells: wt
Treatment: Aldo
Concentration: 0
  Processing cell_treatment folder: wt_Aldo_0
Cells: wt
Treatment: Aldo
Concentration: 1000
  Processing cell_treatment folder: wt_Aldo_1000
Processing date folder: 2024-02-15
Cells: wt
Treatment: Aldo
Concentration: 1000
  Processing cell_treatment folder: wt_Aldo_1000
Cells: wt
Treatment: Aldo
Concentration: 0
  Processing cell_treatment folder: wt_Aldo_0
Cells: wt
Treatment: Aldo
Concentration: AcuteNA
  Processing cell_treatment folder: wt_Aldo_AcuteNA
Processing date folder: 2024-02-29_TEST
Skipping folder with unexpected name format: NT_npchi
Skipping folder with unexpected name format: NT_withchi
Skipping folder with unexpected name format: treated


In [19]:
master_reports

,Mean_fluor_intensity,Mean_tau,Mean_STD,Number_of_objects,Initial_number_of_objects,%_of_object_remained,File Name,cells,treatment,concentration,Date,selected_analysis
0,477.709838,990.556879,20.545257,17,43,39.534884,NT-1,wt,Aldo,0,2024-02-07,analyze_single_fitted_channel
0,373.884800,914.166071,42.204070,8,61,13.114754,NT-2,wt,Aldo,0,2024-02-07,analyze_single_fitted_channel
0,496.942555,945.266398,44.645266,12,23,52.173913,NT-3,wt,Aldo,0,2024-02-07,analyze_single_fitted_channel
0,412.455436,955.685876,28.446394,14,66,21.212121,NT-4,wt,Aldo,0,2024-02-07,analyze_single_fitted_channel
0,380.174888,866.951742,10.541770,6,41,14.634146,NT-5,wt,Aldo,0,2024-02-07,analyze_single_fitted_channel
0,500.435220,967.523189,40.644256,42,64,65.625000,T-1,wt,Aldo,1000,2024-02-07,analyze_single_fitted_channel
0,490.563150,1057.129669,32.839024,87,105,82.857143,T-2,wt,Aldo,1000,2024-02-07,analyze_single_fitted_channel
0,NaN,NaN,NaN,0,3,0.000000,T-3,wt,Aldo,1000,2024-02-07,analyze_single_fitted_channel
0,445.773584,1135.570129,38.117787,20,103,19.417476,T-4,wt,Aldo,1000,2024-02-07,analyze_single_fitted_channel
0,382.868275,943.634880,23.660560,26,75,34.666667,T-5,wt,Aldo,1000,2024-02-07,analyze_single_fitted_channel


In [ ]:
print("Saving master table...")
# Save the master DataFrame as a CSV file
master_results.to_csv(output_folder_path / "master_table.csv")

print("Saving table of reports...")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv(output_folder_path / "master_reports.csv")

print("Processing complete!")